In [1]:
import pandas as pd
import numpy as np
from config import password
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import psycopg2

In [2]:
file = 'nba_payroll.csv'

In [3]:
payroll_df = pd.read_csv(file, encoding="UTF-8")

In [4]:
payroll_df.head()

,Rk,Team,2019-20,2020-21,2021-22,2022-23,2023-24,2024-25
0,1.0,Portland Trail Blazers,"$138,765,934","$109,958,442","$95,713,685","$87,599,310","$89,396,898","$54,250,000"
1,2.0,Milwaukee Bucks,"$135,025,925","$132,270,360","$86,685,346","$71,230,252","$40,396,552",NaN
2,3.0,Miami Heat,"$134,341,791","$82,443,429","$63,800,306","$43,375,416",NaN,NaN
3,4.0,Oklahoma City Thunder,"$133,664,718","$105,489,351","$55,738,605","$7,194,510",NaN,NaN
4,5.0,Los Angeles Clippers,"$133,474,030","$113,852,099","$108,847,187","$13,372,683",NaN,NaN


In [5]:
mapping = {payroll_df.columns[0]:'Rank', payroll_df.columns[1]: 'Team', payroll_df.columns[2]:'2019-20 Payroll'}

payroll_df = payroll_df.rename(columns=mapping)

payroll_df

,Rank,Team,2019-20 Payroll,2020-21,2021-22,2022-23,2023-24,2024-25
0,1.0,Portland Trail Blazers,"$138,765,934","$109,958,442","$95,713,685","$87,599,310","$89,396,898","$54,250,000"
1,2.0,Milwaukee Bucks,"$135,025,925","$132,270,360","$86,685,346","$71,230,252","$40,396,552",NaN
2,3.0,Miami Heat,"$134,341,791","$82,443,429","$63,800,306","$43,375,416",NaN,NaN
3,4.0,Oklahoma City Thunder,"$133,664,718","$105,489,351","$55,738,605","$7,194,510",NaN,NaN
4,5.0,Los Angeles Clippers,"$133,474,030","$113,852,099","$108,847,187","$13,372,683",NaN,NaN
5,6.0,Orlando Magic,"$133,220,515","$122,286,739","$76,234,967","$33,500,000",NaN,NaN
6,7.0,Minnesota Timberwolves,"$132,512,645","$96,033,518","$86,842,098","$77,138,175","$35,970,000",NaN
7,8.0,Golden State Warriors,"$131,527,930","$148,884,768","$158,865,097","$105,855,399","$70,805,665",NaN
8,9.0,Denver Nuggets,"$130,763,093","$107,995,708","$104,503,922","$66,408,837","$36,270,000","$38,610,000"
9,10.0,Washington Wizards,"$130,655,276","$101,202,411","$108,184,168","$90,397,488",NaN,NaN


In [6]:
payroll_df = payroll_df.drop(['Rank'], axis=1)
payroll_df

,Team,2019-20 Payroll,2020-21,2021-22,2022-23,2023-24,2024-25
0,Portland Trail Blazers,"$138,765,934","$109,958,442","$95,713,685","$87,599,310","$89,396,898","$54,250,000"
1,Milwaukee Bucks,"$135,025,925","$132,270,360","$86,685,346","$71,230,252","$40,396,552",NaN
2,Miami Heat,"$134,341,791","$82,443,429","$63,800,306","$43,375,416",NaN,NaN
3,Oklahoma City Thunder,"$133,664,718","$105,489,351","$55,738,605","$7,194,510",NaN,NaN
4,Los Angeles Clippers,"$133,474,030","$113,852,099","$108,847,187","$13,372,683",NaN,NaN
5,Orlando Magic,"$133,220,515","$122,286,739","$76,234,967","$33,500,000",NaN,NaN
6,Minnesota Timberwolves,"$132,512,645","$96,033,518","$86,842,098","$77,138,175","$35,970,000",NaN
7,Golden State Warriors,"$131,527,930","$148,884,768","$158,865,097","$105,855,399","$70,805,665",NaN
8,Denver Nuggets,"$130,763,093","$107,995,708","$104,503,922","$66,408,837","$36,270,000","$38,610,000"
9,Washington Wizards,"$130,655,276","$101,202,411","$108,184,168","$90,397,488",NaN,NaN


In [7]:
# Sort by ascending team name
payroll_df = payroll_df.sort_values('Team')
payroll_df

,Team,2019-20 Payroll,2020-21,2021-22,2022-23,2023-24,2024-25
28,Atlanta Hawks,"$114,377,720","$59,403,929","$58,684,820","$35,377,292",NaN,NaN
22,Boston Celtics,"$121,225,607","$140,567,010","$87,145,456","$76,287,019","$28,601,786",NaN
15,Brooklyn Nets,"$127,379,136","$142,051,521","$137,833,133","$107,900,288",NaN,NaN
17,Charlotte Hornets,"$124,399,147","$80,967,575","$32,889,739","$7,739,116",NaN,NaN
27,Chicago Bulls,"$114,665,634","$106,082,586","$65,249,867","$9,344,636",NaN,NaN
11,Cleveland Cavaliers,"$130,325,319","$109,213,529","$71,168,426","$64,891,181","$6,650,000",NaN
21,Dallas Mavericks,"$121,867,856","$108,221,166","$84,089,411","$63,206,355","$36,016,200",NaN
8,Denver Nuggets,"$130,763,093","$107,995,708","$104,503,922","$66,408,837","$36,270,000","$38,610,000"
10,Detroit Pistons,"$130,445,720","$70,172,303","$42,570,708","$5,539,771",NaN,NaN
7,Golden State Warriors,"$131,527,930","$148,884,768","$158,865,097","$105,855,399","$70,805,665",NaN


In [8]:
payroll_df = payroll_df.reset_index(drop=True)
payroll_df

,Team,2019-20 Payroll,2020-21,2021-22,2022-23,2023-24,2024-25
0,Atlanta Hawks,"$114,377,720","$59,403,929","$58,684,820","$35,377,292",NaN,NaN
1,Boston Celtics,"$121,225,607","$140,567,010","$87,145,456","$76,287,019","$28,601,786",NaN
2,Brooklyn Nets,"$127,379,136","$142,051,521","$137,833,133","$107,900,288",NaN,NaN
3,Charlotte Hornets,"$124,399,147","$80,967,575","$32,889,739","$7,739,116",NaN,NaN
4,Chicago Bulls,"$114,665,634","$106,082,586","$65,249,867","$9,344,636",NaN,NaN
5,Cleveland Cavaliers,"$130,325,319","$109,213,529","$71,168,426","$64,891,181","$6,650,000",NaN
6,Dallas Mavericks,"$121,867,856","$108,221,166","$84,089,411","$63,206,355","$36,016,200",NaN
7,Denver Nuggets,"$130,763,093","$107,995,708","$104,503,922","$66,408,837","$36,270,000","$38,610,000"
8,Detroit Pistons,"$130,445,720","$70,172,303","$42,570,708","$5,539,771",NaN,NaN
9,Golden State Warriors,"$131,527,930","$148,884,768","$158,865,097","$105,855,399","$70,805,665",NaN


In [9]:
payroll_df = payroll_df.drop(payroll_df.index[[30]])

payroll_df

,Team,2019-20 Payroll,2020-21,2021-22,2022-23,2023-24,2024-25
0,Atlanta Hawks,"$114,377,720","$59,403,929","$58,684,820","$35,377,292",NaN,NaN
1,Boston Celtics,"$121,225,607","$140,567,010","$87,145,456","$76,287,019","$28,601,786",NaN
2,Brooklyn Nets,"$127,379,136","$142,051,521","$137,833,133","$107,900,288",NaN,NaN
3,Charlotte Hornets,"$124,399,147","$80,967,575","$32,889,739","$7,739,116",NaN,NaN
4,Chicago Bulls,"$114,665,634","$106,082,586","$65,249,867","$9,344,636",NaN,NaN
5,Cleveland Cavaliers,"$130,325,319","$109,213,529","$71,168,426","$64,891,181","$6,650,000",NaN
6,Dallas Mavericks,"$121,867,856","$108,221,166","$84,089,411","$63,206,355","$36,016,200",NaN
7,Denver Nuggets,"$130,763,093","$107,995,708","$104,503,922","$66,408,837","$36,270,000","$38,610,000"
8,Detroit Pistons,"$130,445,720","$70,172,303","$42,570,708","$5,539,771",NaN,NaN
9,Golden State Warriors,"$131,527,930","$148,884,768","$158,865,097","$105,855,399","$70,805,665",NaN


In [10]:
payroll_df = payroll_df.drop(columns=['2020-21', '2021-22', '2022-23', '2023-24', '2024-25'])

payroll_df

,Team,2019-20 Payroll
0,Atlanta Hawks,"$114,377,720"
1,Boston Celtics,"$121,225,607"
2,Brooklyn Nets,"$127,379,136"
3,Charlotte Hornets,"$124,399,147"
4,Chicago Bulls,"$114,665,634"
5,Cleveland Cavaliers,"$130,325,319"
6,Dallas Mavericks,"$121,867,856"
7,Denver Nuggets,"$130,763,093"
8,Detroit Pistons,"$130,445,720"
9,Golden State Warriors,"$131,527,930"


In [11]:
payroll_df['team_id'] = np.arange(len(payroll_df))

payroll_df

,Team,2019-20 Payroll,team_id
0,Atlanta Hawks,"$114,377,720",0
1,Boston Celtics,"$121,225,607",1
2,Brooklyn Nets,"$127,379,136",2
3,Charlotte Hornets,"$124,399,147",3
4,Chicago Bulls,"$114,665,634",4
5,Cleveland Cavaliers,"$130,325,319",5
6,Dallas Mavericks,"$121,867,856",6
7,Denver Nuggets,"$130,763,093",7
8,Detroit Pistons,"$130,445,720",8
9,Golden State Warriors,"$131,527,930",9


In [12]:
payroll_df = payroll_df.rename(columns={"Team": "team", "2019-20 Payroll": "payroll"})

payroll_df

,team,payroll,team_id
0,Atlanta Hawks,"$114,377,720",0
1,Boston Celtics,"$121,225,607",1
2,Brooklyn Nets,"$127,379,136",2
3,Charlotte Hornets,"$124,399,147",3
4,Chicago Bulls,"$114,665,634",4
5,Cleveland Cavaliers,"$130,325,319",5
6,Dallas Mavericks,"$121,867,856",6
7,Denver Nuggets,"$130,763,093",7
8,Detroit Pistons,"$130,445,720",8
9,Golden State Warriors,"$131,527,930",9


In [20]:
payroll_df['payroll'] = payroll_df['payroll'].str.replace(',', '')
payroll_df['payroll'] = payroll_df['payroll'].str.replace('$', '')

In [21]:
connection_string = f"postgres:{password}@localhost:5432/teamvalue_db"
engine = create_engine(f'postgresql://{connection_string}')

In [22]:
engine.table_names()

['avgattendance', 'payroll']

In [23]:
payroll_df.to_sql(name='payroll', con=engine, if_exists='append', index=False)